In [1]:
from utils.utils_IO import load_object, write_video
from utils.utils_plotting import plot_cams_and_points
from copy import deepcopy
from pathlib import Path
import numpy as np

load_dict = load_object("sawtell_points.pickle")
config = load_dict["config"]
cam_group = load_dict["cam_group"]
points_3d = load_dict["points_3d"]
bp_names = config["bp_names"]
cam_names = ["Top", "Front", "Side"]

In [8]:
frame_i = 256
slice_3d = points_3d[frame_i]

scene_camera_start = {
    "center": {
        "x": -0.16674291292903914,
        "y": 0.07518956758153557,
        "z": 0.04021290485414575,
    },
    "eye": {
        "x": 0.10230527610492485,
        "y": -0.25113595274977013,
        "z": -1.484742818689902,
    },
    "up": {
        "x": -0.002160309878332354,
        "y": -0.9779372269510715,
        "z": 0.2088878005161597,
    },
}

skeleton_bp = {}
for i, bp in enumerate(bp_names):
    skeleton_bp[bp] = points_3d[frame_i, i, :]
skeleton_lines = config.skeleton

skel_fig = plot_cams_and_points(
    cam_group=cam_group,
    points_3d=slice_3d,
    point_size=3,
    skeleton_bp=skeleton_bp,
    skeleton_lines=skeleton_lines,
    font_size=20,
    point_colors=config.color_list,
    cam_names=cam_names,
    show_plot=False,
)

skel_fig_layout = deepcopy(skel_fig)
skel_fig_layout.update_layout(
    scene_camera=scene_camera_start,
    scene=dict(
        xaxis=dict(
            showticklabels=False,
            range=[
               np.nanmin(points_3d[:, 0]),
               np.nanmax(points_3d[:, 0]) / 4,
            ],
        ),
        yaxis=dict(
            showticklabels=False,
            range=[
               np.nanmin(points_3d[:, 1]),
               np.nanmax(points_3d[:, 1]) / 4,
            ],
        ),
        zaxis=dict(
            showticklabels=False,
            range=[
               np.nanmin(points_3d[:, 2]),
               np.nanmax(points_3d[:, 2]),
            ],
        ),
    ),
)
skel_fig_layout["layout"]["scene"]["aspectmode"] = "cube"
skel_fig["layout"] = skel_fig_layout["layout"]
skel_fig.write_image("fish_with_cams.eps", scale=5)


In [4]:
num_frames = 200
downsampling = 5
start_frame = 256
save_frames_dir = Path("./fish_frames")
save_frames_dir.mkdir(exist_ok=True, parents=True)


scene_camera_start = {
    "center": {
        "x": -0.16674291292903914,
        "y": 0.07518956758153557,
        "z": 0.04021290485414575,
    },
    "eye": {
        "x": 0.10230527610492485,
        "y": -0.25113595274977013,
        "z": -1.484742818689902,
    },
    "up": {
        "x": -0.002160309878332354,
        "y": -0.9779372269510715,
        "z": 0.2088878005161597,
    },
}

scene_camera_end = {
    "center": {
        "x": -0.17370989580811724,
        "y": 0.26159536778553444,
        "z": -0.0009052882205128804,
    },
    "eye": {
        "x": -0.08807379683322851,
        "y": 0.15772830717656666,
        "z": -0.4862877014701171,
    },
    "up": {
        "x": -0.002160309878332347,
        "y": -0.9779372269510711,
        "z": 0.20888780051615963,
    },
}

for i in range(num_frames):
    frame_i = start_frame + i * downsampling
    slice_3d = points_3d[frame_i]

    skeleton_bp = {}
    for bp_idx, bp in enumerate(bp_names):
        skeleton_bp[bp] = points_3d[frame_i, bp_idx, :]
    skeleton_lines = config.skeleton

    skel_fig = plot_cams_and_points(
        cam_group=cam_group,
        points_3d=slice_3d,
        point_size=3,
        skeleton_bp=skeleton_bp,
        skeleton_lines=skeleton_lines,
        font_size=20,
        point_colors=config.color_list,
        cam_names=cam_names,
        show_plot=False,
    )

    curr_scene_camera = {}
    for key in scene_camera_start.keys():
        curr_scene_camera[key] = {}
        for coord in scene_camera_start[key].keys():
            curr_scene_camera[key][coord] = scene_camera_start[key][coord] + (
                scene_camera_end[key][coord] - scene_camera_start[key][coord]
            ) * (i / num_frames)

    curr_scene_camera
    curr_scene_camera["projection"] = {"type": "perspective"}

    skel_fig.update_layout(
        scene_camera=curr_scene_camera,
        scene=dict(
            xaxis=dict(
                showticklabels=False,
            ),
            yaxis=dict(
                showticklabels=False,
            ),
            zaxis=dict(
                showticklabels=False,
            ),
        ),
    )
    skel_fig.update_scenes(aspectmode="cube")
    skel_fig.write_image(str(save_frames_dir / f"{i}.png"), scale=5)

write_video(
    image_dir=str(save_frames_dir), out_file=str(save_frames_dir / "fish_video.mov")
)


In [ ]:
skel_fig.layout.scene.camera